In [1]:
import cheetah.bmad

In [2]:
os.environ["LCLS_LATTICE"] = str((Path(".").absolute().parent / "lcls-lattice"))
os.environ["LCLS_LATTICE"]

'/Users/jankaiser/Documents/DESY/lcls-lattice'

In [11]:
lattice_file_path = (
    Path("$LCLS_LATTICE") / "bmad" / "models" / "cu_hxr" / "cu_hxr.lat.bmad"
)
lattice_file_path

PosixPath('$LCLS_LATTICE/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [4]:
resolved_lattice_file_path = Path(
    *[
        os.environ[part[1:]] if part.startswith("$") else part
        for part in lattice_file_path.parts
    ]
)
resolved_lattice_file_path

PosixPath('/Users/jankaiser/Documents/DESY/lcls-lattice/bmad/models/cu_hxr/cu_hxr.lat.bmad')

In [22]:
def read_clean_lattice_file(lattice_file_path: Path) -> list[str]:
    with open(lattice_file_path) as f:
        lines = f.readlines()

    # Remove comments and empty lines
    lines = [line.strip() for line in lines]
    # Remove comments (i.e. all characters after a '!')
    lines = [re.sub(r"!.*", "", line) for line in lines]
    # Remove empty lines
    lines = [line for line in lines if line]

    # Replace lines calling external files with the lines of the external file
    replaced_lines = []
    for i, line in enumerate(lines):
        if line.startswith("call, file ="):
            external_file_path = Path(line.split("=")[1].strip())
            resolved_external_file_path = Path(
                *[
                    os.environ[part[1:]] if part.startswith("$") else part
                    for part in external_file_path.parts
                ]
            )
            if not resolved_external_file_path.is_absolute():
                resolved_external_file_path = (
                    lattice_file_path.parent / resolved_external_file_path
                )
            external_file_lines = read_clean_lattice_file(resolved_external_file_path)
            replaced_lines += external_file_lines
        else:
            replaced_lines.append(line)

    # Make lines all lower case (done late because environment variables are case sensitive)
    replaced_lines = [line.lower() for line in replaced_lines]

    return replaced_lines

In [23]:
lines = read_clean_lattice_file(resolved_lattice_file_path)
lines

['beginning[beta_a] =  5.91253676811640894e+000',
 'beginning[alpha_a] =  3.55631307633660354e+000',
 'beginning[beta_b] =  5.91253676811640982e+000',
 'beginning[alpha_b] =  3.55631307633660398e+000',
 'beginning[e_tot] = 6e6',
 'parameter[geometry] = open',
 'parameter[particle] = electron',
 'beginning[theta_position] = -35*pi/180',
 'beginning[z_position] = 3050.512000 - 1032.60052',
 'beginning[x_position] = 10.44893',
 'setsp = 0',
 'setcus = 0',
 'setal = 0',
 'setda = 0',
 'setxleap2 = 0 ',
 'sethxrss = 0 ',
 'setsxrss = 0 ',
 'setcbxfel = 0 ',
 'setpepx = 0 ',
 'intgsx = 30.0 ',
 'intghx = 30.0 ',
 'cb = 1.0e10/c_light ',
 'gev2mev = 1000.0        ',
 'in2m = 0.0254        ',
 'mc2 = 510.99906e-6  ',
 'injdeg = -35.0        ',
 'zoffinj = 0.012100   ',
 'xoff = -25.610*in2m ',
 'yoff = 25.570*in2m ',
 'emitxn = 1.00e-06 ',
 'emityn = 1.00e-06 ',
 'bleng = 0.83e-03 ',
 'esprd = 2.00e-05 ',
 'e00 = 0.006 ',
 'e0i = 0.064 ',
 'ei = 0.135 ',
 'ebc1 = 0.220 ',
 'ebc2 = 5.000 ',
 'e